## Импорт модулей

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from nltk.stem import wordnet
import statistics
import nltk
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import pos_tag, word_tokenize
from sklearn.metrics import pairwise_distances 
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import torch.nn.functional as F
from sklearn.ensemble import RandomForestClassifier

In [2]:
# nltk.download()

In [3]:
morph = MorphAnalyzer()
stop_words = stopwords.words("russian")

## Парсинг сайтов

функции для упрощение парсинга

In [4]:
def openURL(url):
    page = requests.get(url)
    print(page.status_code)
    soup = BeautifulSoup(page.text, "lxml")
    return(soup)

def filter(soup, tag, class_):
    text = []
    textF = []
    text = soup.findAll(tag, class_)
    for data in text:
        textF.append(data.text)
    return(textF)

def createDF(Q, A):
    data = pd.DataFrame(columns=['Question', 'Answer'])
    data['Question'] = Q
    data['Answer'] = A
    return(data)

Раздел FAQ с сайта РУДН

In [5]:
# soup = openURL('https://www.rudn.ru/education/learning-quality-system/faq')
# Q = filter(soup, 'span', 'acc-header-text')
# A = filter(soup, 'div', 'WYSIWYG')
# data_RUDN = createDF(Q, A)

Раздел FAQ c сайта фитнесс клуба Spirit

In [6]:
# soup = openURL('https://spiritfit.ru/')
# Q = filter(soup, 'div', 'b-accordion__heading')
# A = filter(soup, 'div', 'b-accordion__content')
# data_Spirit = createDF(Q, A)

Раздел FAQ с сайта медиа-платформы Premier

In [7]:
# soup = openURL('https://premier.one/info/tntp/faq')
# QA = filter(soup, 'li', 'faq-list__item')
# data_Premier = pd.DataFrame(columns=['Question', 'Answer'])
# dt = pd.DataFrame(QA)
# A = []

# for i in range(len(dt)):
#     A.append(dt.loc[i, 0].split('\n\n'))

# for i in range(len(A)):
#     data_Premier.loc[i, 'Question'] = A[i][0]
#     A[i][0] = ''
#     data_Premier.loc[i, 'Answer'] = A[i]

Раздел FAQ с сайта Мосгортранс

In [8]:
# soup = openURL('https://transport.mos.ru/help/faq')
# QA = filter(soup, 'div', 'h3mb')
# data_Mos = pd.DataFrame(columns=['Question', 'Answer'])
# dt = pd.DataFrame(QA)
# A = []

# for i in range(len(dt)):
#     A.append(dt.loc[i, 0].split('\n\n'))

# for i in range(1, len(QA), 2):
#     data_Mos.loc[i, 'Question'] = QA[i]
#     data_Mos.loc[i, 'Answer'] = QA[i+1]

# data_Mos = data_Mos.reset_index()
# data_Mos = data_Mos.drop(['index'], axis=1)


Сохраняю датасеты в csv файлы, так как иногда сайты закрывают доступ, или меняют ссылки.

In [9]:
# data_RUDN.to_csv('data_RUDN.csv', encoding='utf-8-sig', sep=',',index=None)
# data_Spirit.to_csv('data_Spirit.csv', encoding='utf-8-sig', sep=',',index=None)
# data_Premier.to_csv('data_Premier.csv', encoding='utf-8-sig', sep=',',index=None)
# data_Mos.to_csv('data_Mos.csv', encoding='utf-8-sig', sep=',',index=None)

In [10]:
data_RUDN = pd.read_csv('data_RUDN.csv')
data_Spirit = pd.read_csv('data_Spirit.csv')
data_Premier = pd.read_csv('data_Premier.csv')
data_Mos = pd.read_csv('data_Mos.csv')

## Нормализация вопросов

Функция для удаления лишних тегов, таких как \n, \t, \p, и т.д.

In [11]:
def norm_qa(data):
    dt = re.sub('<[^<>]*>', '', data).replace('\n', '').replace('\r', '').replace('\t', '')
    return dt

функция, удаляет все символы, кроме букв и цифр, приводит их к строчному написанию, а затем токенизирует их.

In [12]:
def norm_token(data):
    words = []
    dt = re.sub(r'\n', ' ', str(data).lower())
    dt = re.sub(r'[^а-яёa-z0-9]', ' ', str(dt))
    dt_tokens = word_tokenize(dt)
    for token in dt_tokens:
        if token not in stop_words:
            token = morph.normal_forms(token)[0]
            words.append(token)
    return words 

вспомогательная функция, которая возвращает в виде текста, а не токенов, что необходимо в некоторых моделях.

In [13]:
def norm(data):
    return " ".join(data)

Применяю нормализацию ко всем датасетам

In [14]:
data_RUDN['Question'] = data_RUDN['Question'].apply(norm_qa)
data_RUDN['Answer'] = data_RUDN['Answer'].apply(norm_qa)

data_Spirit['Question'] = data_Spirit['Question'].apply(norm_qa)
data_Spirit['Answer'] = data_Spirit['Answer'].apply(norm_qa)

data_Premier['Question'] = data_Premier['Question'].apply(norm_qa)
data_Premier['Answer'] = data_Premier['Answer'].apply(norm_qa)

data_Mos['Question'] = data_Mos['Question'].apply(norm_qa)
data_Mos['Answer'] = data_Mos['Answer'].apply(norm_qa)

data_RUDN['token_lemmaQ']=data_RUDN['Question'].apply(norm_token)
data_Spirit['token_lemmaQ']=data_Spirit['Question'].apply(norm_token)
data_Premier['token_lemmaQ']=data_Premier['Question'].apply(norm_token)
data_Mos['token_lemmaQ']=data_Mos['Question'].apply(norm_token)

data_RUDN['token_lemmaA']=data_RUDN['Answer'].apply(norm_token)
data_Spirit['token_lemmaA']=data_Spirit['Answer'].apply(norm_token)
data_Premier['token_lemmaA']=data_Premier['Answer'].apply(norm_token)
data_Mos['token_lemmaA']=data_Mos['Answer'].apply(norm_token)

data_RUDN['lemmaQ']=data_RUDN['token_lemmaQ'].apply(norm)
data_Spirit['lemmaQ']=data_Spirit['token_lemmaQ'].apply(norm)
data_Premier['lemmaQ']=data_Premier['token_lemmaQ'].apply(norm)
data_Mos['lemmaQ']=data_Mos['token_lemmaQ'].apply(norm)

data_RUDN['lemmaA']=data_RUDN['token_lemmaA'].apply(norm)
data_Spirit['lemmaA']=data_Spirit['token_lemmaA'].apply(norm)
data_Premier['lemmaA']=data_Premier['token_lemmaA'].apply(norm)
data_Mos['lemmaA']=data_Mos['token_lemmaA'].apply(norm)


## Модель на основе TF-IDF

In [15]:
tfidf = TfidfVectorizer()

def chat_tfidf(data, text):
    x_tfidf = tfidf.fit_transform(data['lemmaQ']).toarray()
    df_tfidf = pd.DataFrame(x_tfidf, columns=tfidf.get_feature_names_out())
    tf = tfidf.transform([norm(norm_token(text))]).toarray()
    cos = 1-pairwise_distances(df_tfidf, tf, metric='cosine')
    return data['Answer'].loc[cos.argmax()].lstrip()

## Модель на основе Bag of Words

In [16]:
cv = CountVectorizer()

def chat_bow(data, text):
    X = cv.fit_transform(data['lemmaQ']).toarray()
    features = cv.get_feature_names_out()
    df_bow = pd.DataFrame(X, columns = features)
    lemma = norm(norm_token(text))
    bow = cv.transform([lemma]).toarray()
    cosine_value = 1 - pairwise_distances(df_bow, bow, metric='cosine')
    index_value = cosine_value.argmax()
    return data['Answer'].loc[index_value].lstrip()


## Модель на основе классификатора

In [17]:
def chat_classifier(data, question):
    # разделение на признаки и метки класса
    X = data["lemmaQ"].tolist()
    y = []
    for i in range(len(data)):
        y.append(i)

    # векторизация вопросов
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(X)

    # обучение модели классификации
    classifier = RandomForestClassifier()
    classifier.fit(X, y)

    # функция для предсказания ответа на вопрос пользователя
    def predict_answer(question):
        question_vectorized = vectorizer.transform([question])
        answer = classifier.predict(question_vectorized)[0]
        return answer

    # использование функции для предсказания ответа на вопрос пользователя
    answer = predict_answer(norm(norm_token(question)))

    return data['Answer'].loc[answer].lstrip()

## Тестирование

Добавляю в каждый датасет по 20 вопросов от пользователя.

In [18]:
def user_question_input(data):
    l = len(data)//20
    for i in range(0, len(data), l):
        print(i)
        print(data['Question'][i])
        data.loc[i, 'User_question'] = input()

In [19]:
# user_question_input(data_RUDN)
# print()
# user_question_input(data_Spirit)
# print()
# user_question_input(data_Premier)
# print()
# user_question_input(data_Mos)

In [20]:
# data_RUDN_test = data_RUDN.dropna()
# data_Spirit_test = data_Spirit.dropna()
# data_Premier_test = data_Premier.dropna()
# data_Mos_test = data_Mos.dropna()

In [21]:
# data_RUDN_test.to_csv('data_RUDN_test.csv', encoding='utf-8-sig', sep=',',index=None)
# data_Spirit_test.to_csv('data_Spirit_test.csv', encoding='utf-8-sig', sep=',',index=None)
# data_Premier_test.to_csv('data_Premier_test.csv', encoding='utf-8-sig', sep=',',index=None)
# data_Mos_test.to_csv('data_Mos_test.csv', encoding='utf-8-sig', sep=',',index=None)

In [22]:
data_RUDN_test = pd.read_csv('data_RUDN_test.csv')
data_Spirit_test = pd.read_csv('data_Spirit_test.csv')
data_Premier_test = pd.read_csv('data_Premier_test.csv')
data_Mos_test = pd.read_csv('data_Mos_test.csv')

In [23]:
def chat_test(datasets):
    cnt = [0, 0, 0, 0]
    e_cnt = [0, 0, 0, 0]
    e_cnt_tfidf = [0, 0, 0, 0]
    e_cnt_bow = [0, 0, 0, 0]
    e_cnt_classifier = [0, 0, 0, 0]
    for dataset_idx, data in enumerate(datasets):
        for i in range(len(data)):
            tfidf = chat_tfidf(data, data['User_question'][i])
            bow = chat_bow(data, data['User_question'][i])
            classifier = chat_classifier(data, data['User_question'][i])

            if (tfidf == bow == classifier == data['Answer'][i].lstrip()):
                cnt[dataset_idx] += 1
            else:
                e_cnt[dataset_idx] += 1
                if (tfidf == data['Answer'][i]):
                    e_cnt_tfidf[dataset_idx] += 1
                elif (bow == data['Answer'][i]):
                    e_cnt_bow[dataset_idx] += 1
                elif (classifier == data['Answer'][i]):
                    e_cnt_classifier[dataset_idx] += 1

    tfidf_accuracy = [round((cnt[i]+e_cnt_tfidf[i])/(e_cnt[i]+cnt[i]), 2) for i in range(len(datasets))]
    bow_accuracy = [round((cnt[i]+e_cnt_bow[i])/(e_cnt[i]+cnt[i]), 2) for i in range(len(datasets))]
    classifier_accuracy = [round((cnt[i]+e_cnt_classifier[i])/(e_cnt[i]+cnt[i]), 2) for i in range(len(datasets))]

    return tfidf_accuracy, bow_accuracy, classifier_accuracy

In [24]:
datasets = [data_RUDN_test, data_Spirit_test, data_Premier_test, data_Mos_test]
tfidf_accuracy, bow_accuracy, classifier_accuracy = chat_test(datasets)

print("TFIDF accuracy:", statistics.mean(tfidf_accuracy))
print("BoW accuracy:", statistics.mean(bow_accuracy))
print("Classifier accuracy:", statistics.mean(classifier_accuracy))

TFIDF accuracy: 0.9199999999999999
BoW accuracy: 0.875
Classifier accuracy: 0.9025


In [25]:
data_Mos

,Question,Answer,token_lemmaQ,token_lemmaA,lemmaQ,lemmaA
0,Что такое «ковидная амнистия»?,С 1 ию...,"[такой, ковидный, амнистия]","[1, июнь, житель, индивидуальный, предпринимат...",такой ковидный амнистия,1 июнь житель индивидуальный предприниматель ю...
1,Как я могу вернуть деньги за уже оплаченный шт...,Для оформления компенсации необходимо обратит...,"[мочь, вернуть, деньга, оплатить, штраф]","[оформление, компенсация, необходимо, обратить...",мочь вернуть деньга оплатить штраф,оформление компенсация необходимо обратиться ц...
2,"Где я могу получить копию постановления, если ...",Если постановление выписано контролером ГКУ «О...,"[мочь, получить, копия, постановление, оно, вы...","[постановление, выписать, контролёр, гку, орга...",мочь получить копия постановление оно выписать...,постановление выписать контролёр гку организат...
3,Как быстро мне вернут деньги за оплаченный штр...,Обраще...,"[быстро, вернуть, деньга, оплатить, штраф]","[обращение, регистрироваться, сотрудник, день,...",быстро вернуть деньга оплатить штраф,обращение регистрироваться сотрудник день пода...
4,"Что делать, если вещи упали на рельсы?","Если Ваши вещи упали на рельсы, следует незаме...","[делать, вещь, упасть, рельс]","[ваш, вещь, упасть, рельс, следовать, незамедл...",делать вещь упасть рельс,ваш вещь упасть рельс следовать незамедлительн...
...,...,...,...,...,...,...
361,Как дезинфицируют метро?,"На всех станциях метро дезинфицируют двери, по...","[дезинфицировать, метро]","[станция, метро, дезинфицировать, дверь, поруч...",дезинфицировать метро,станция метро дезинфицировать дверь поручень к...
362,Как дезинфицируют наземный транспорт?,"В конце смены весь транспорт, который работает...","[дезинфицировать, наземный, транспорт]","[конец, смена, весь, транспорт, который, работ...",дезинфицировать наземный транспорт,конец смена весь транспорт который работать ма...
363,Как дезинфицируют железнодорожный транспорт?,Влажную уборку ж/д транспорта проводят дважды ...,"[дезинфицировать, железнодорожный, транспорт]","[влажный, уборка, далее, транспорт, проводить,...",дезинфицировать железнодорожный транспорт,влажный уборка далее транспорт проводить дважд...
364,"Что делать, если я заметил грязный автомобиль ...",Все компании такси и каршеринга проинформирова...,"[делать, заметить, грязный, автомобиль, такси,...","[компания, такси, каршеринг, проинформировать,...",делать заметить грязный автомобиль такси карше...,компания такси каршеринг проинформировать необ...


## Связь с ботом

In [26]:
import telebot;
from telebot import types

In [27]:
bot = telebot.TeleBot('TOKEN')

@bot.message_handler(commands=['start'])
def send_welcome(message):
    global data
    data = None

    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("РУДН")
    btn2 = types.KeyboardButton("Premier")
    btn3 = types.KeyboardButton("Spirit")
    btn4 = types.KeyboardButton("Мосгортранс")
    markup.add(btn1, btn2, btn3, btn4)

    bot.send_message(message.chat.id, f"Привет, {message.from_user.first_name}!\
                    \nЯ бот, готовый помочь Вам ответить на ваши вопросы.\
                      \n\nПожалуйста, выберите датасет:", reply_markup=markup)
    bot.register_next_step_handler(message, data_choice)

def data_choice(message):
    global data
    datasets = {
        "РУДН": data_RUDN,
        "Premier": data_Premier,
        "Spirit": data_Spirit,
        "Мосгортранс": data_Mos
    }
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("РУДН")
    btn2 = types.KeyboardButton("Premier")
    btn3 = types.KeyboardButton("Spirit")
    btn4 = types.KeyboardButton("Мосгортранс")
    markup.add(btn1, btn2, btn3, btn4)

    if message.text not in datasets:
        bot.send_message(message.chat.id, "Ошибка, пожалуйста, выберите датасет из предложенных вариантов.",\
                          reply_markup=markup)
        bot.register_next_step_handler(message, data_choice)
        return

    data = datasets[message.text]

    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Назад")
    markup.add(btn1)

    bot.send_message(message.chat.id, f"Вы выбрали датасет {message.text}. \nВведите свой вопрос:",\
                      reply_markup=markup)
    bot.register_next_step_handler(message, handle_question)

def handle_question(message):
    global data

    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Назад")
    markup.add(btn1)

    if message.text == "/start" or message.text.lower() == "назад":
        send_welcome(message)
        return
    else:
        answer = chat_tfidf(data, norm(norm_token(message)))
        bot.send_message(message.chat.id, answer, reply_markup=markup)

    bot.register_next_step_handler(message, handle_question)

bot.polling(none_stop=True, interval=0)

In [36]:
data_RUDN[['Question', 'Answer']][:5]

,Question,Answer
0,Существуют ли в РУДН подготовительные курсы дл...,"В РУДН есть Управление довузовской подготовки,..."
1,Что преподают на подготовительных курсах РУДН?,В состав Управления довузовской подготовки вхо...
2,Я хочу поступать в РУДН.,"Информация о приёме на обучение, доступна по с..."
3,Какие существуют в РУДН особенности приема по ...,Победа в заключительном этапе Всероссийской ол...
4,Какие нужно документы для поступления в РУДН?,Список необходимых документов для подачи при п...
